In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install numpy scipy pandas matplotlib numpy_ringbuffer sklearn

import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
sys.path.append('..')

In [ ]:
# currencies = ['usd', 'btc', 'eth', 'ltc', 'xrp', 'eos']
# pairs = [c + '_usd' for c in currencies if c != 'usd']
# volume_keys = [c + '_tx_volume' for c in currencies if c != 'usd']

# def prep_data(file):
#     data = pickle.load(open(file, 'rb'))
#     dates = [x['date'] for x in data]
#     prices = [{k:v for k,v in x.items() if k in pairs} for x in data]
#     volumes = [{(k.partition('_')[0] + '_usd'):v for k,v in x.items() if k in volume_keys} for x in data]
#     return {
#         'prices': pd.DataFrame(prices, index = dates),
#         'volumes': pd.DataFrame(volumes, index = dates)
#     }

# def reduce_data(data, resampling):
#     '''Averages prices, sums volumes'''
#     prices = data['prices'].resample(resampling).first().fillna(method='ffill')
#     volumes = data['volumes'].resample(resampling).sum().fillna(method='ffill')
#     return { 'prices': prices, 'volumes': volumes }

# def tail_data(data, n):
#     '''get the last n points of the given data'''
#     prices = data['prices'].tail(n)
#     volumes = data['volumes'].tail(n)
#     return { 'prices': prices, 'volumes': volumes }

# def viz_data(data):
#     '''Only plots prices for now'''
#     plt.plot(data['prices'] / data['prices'].mean() - 1)
#     plt.show()

# def find_gaps(data, freq):
#     idx_ref = pd.date_range(start=data.index[0], end=data.index[-1],freq=freq)
#     gaps = idx_ref[~idx_ref.isin(data.index)]
#     return gaps

# data = prep_data('data/data.p')
# data_min = reduce_data(prep_data('data/data-minute.p'), '1Min')
# data_5min = reduce_data(data_min, '5Min')
# data_15min = reduce_data(data_min, '15Min')
# viz_data(data_min)

In [ ]:
data_min = pd.read_hdf('data/1min.h5')
data_15min = data_min.resample('15Min').first()
data_1h = data_min.resample('1h').first()

In [ ]:
price_data = data_15min.apply(lambda x: x['price'])
(price_data / price_data.mean()).plot()

In [ ]:
# def plot_matrix(df,size=10):
#     '''Function plots a graphical correlation matrix for each pair of columns in the dataframe.

#     Input:
#         df: pandas DataFrame
#         size: vertical and horizontal size of the plot'''

#     fig, ax = plt.subplots(figsize=(size, size))
#     ax.matshow(df)
#     plt.xticks(range(len(df.columns)), df.columns);
#     plt.yticks(range(len(df.index)), df.index);
#     # Loop over data dimensions and create text annotations.
#     for i in range(len(df.index)):
#         for j in range(len(df.columns)):
#             ax.text(j, i, '{:0.2f}'.format(df.iloc[i, j]), ha="center", va="center", color="w")

# plot_matrix(data_15min['prices'].corr())

In [ ]:
# def cross_correlate_(x, y):
#     return np.argmax(np.correlate(x, y, mode='full')) - len(x) + 1

# def cross_correlate(df):
#     '''Compute cross-correlation matrix for the given dataframe.'''
#     ccs = pd.DataFrame(index=df.columns, columns=df.columns)
#     for i in df.columns:
#         for j in df.columns:
#             if i == j:
#                 ccs.loc[i,j] = 0
#                 continue
#             if np.isnan(ccs.loc[i,j]):
#                 ccs.loc[i,j] = cross_correlate_(df[i], df[j])
#                 ccs.loc[j,i] = -ccs.loc[i,j]
#     return ccs

# print(cross_correlate(pd.DataFrame([[1,2],[2,1],[1,2],[2,1],[1,2]])))
# print(cross_correlate(pd.DataFrame([[1,1],[2,2],[3,3],[4,4],[5,5]])))
    
# print(cross_correlate(data_min['prices']))

In [ ]:
##### import numpy as np
import pandas as pd
from trader.util.stats import Gaussian

# Note: Assumes all orders fill at last trade price. Attempting to simulate market-making would
# require combing through book and trade data, which is too much work for us to do at the moment.

def data_currencies(data):
    quote_currency = data.iloc[0].index[0].partition("_")[2]
    currencies = [quote_currency]
    for pair in data.iloc[0].index:
        currencies.append(pair.partition('_')[0])
    return currencies

def get_orders(balances, prices, fairs, size, fees, min_edge):
    '''Given current balances, prices, and fair estimates, determine which orders to place.
    Assumes all pairs are XXX_USD.
    `fairs` should be a Gaussian type. '''
    quote_currency = prices.index[0].partition("_")[2]
    
    gradient = fairs.gradient(prices) * fairs.mean
    balance_direction_vector = gradient / (np.linalg.norm(gradient) + 1e-100)
    target_balance_values = balance_direction_vector * fairs.z_score(prices) * size
    pair_balances = balances.drop([quote_currency]).rename(lambda c: '{}_{}'.format(c, quote_currency))
    proposed_orders = target_balance_values / prices - pair_balances
    profitable = np.sign(proposed_orders) * (fairs.mean / prices - 1) > fees + min_edge
    profitable_orders = proposed_orders * profitable
    
#     unprofitable_balance = pair_balances * edge.mean < 0
#     # TODO: in the near future we should model this via the cdf of the profitable order cutoff
#     p_reversion = 0.6
#     profitability_gross_edge_cutoff = fees + edge.stddev * min_edge
#     positive_ev_from_closing_position = p_reversion * (gross_edge.mean - fees) > (1-p_reversion) * (profitability_gross_edge_cutoff - fees)
#     balance_closing_orders = -pair_balances * unprofitable_balance * positive_ev_from_closing_position

    return profitable_orders # + balance_closing_orders


def execute_orders(fees, prices, balances, orders):
    for (pair, size) in orders.items():
        currency, quote_currency = pair.split("_")
        value = size * prices[pair]
        balances[quote_currency] -= value
        balances[quote_currency] -= abs(value) * fees
        balances[currency] += size


def run(strategy, data, size=1000, fees=0, min_edge = 0):
    balances = pd.Series(dict.fromkeys(data_currencies(data), 0.))
    balances_ = []
    fairs_ = []
    for frame in data:
        fairs = strategy.step(frame)
        orders = get_orders(balances, frame['price'], fairs, size, fees, min_edge)
        execute_orders(fees, frame['price'], balances, orders)

        fairs_.append(fairs)
        balances_.append(balances.copy())
    return {
        'data': data,
        'fairs': pd.DataFrame(fairs_, index=data.index),
        'balances': pd.DataFrame(balances_, index=data.index)
    }


In [ ]:
from strategy import HoldStrategy
from execution import analyze

analyze(run(HoldStrategy(), data_min.tail(10)))

In [ ]:
from strategy import KalmanFilterStrategy

analyze(run(KalmanFilterStrategy(correlation_window_size = 64, movement_half_life = 3), data_15min, fees=0.002))

In [ ]:
from strategy import Strategy
from trader.util.stats import Ema, Gaussian
from execution import analyze

import numpy as np
import pandas as pd
from numpy_ringbuffer import RingBuffer
from statsmodels.tsa.stattools import coint

def coint(df):
    ratios = pd.DataFrame(0, index=df.columns, columns=df.columns)
    p_values = pd.DataFrame(0, index=df.columns, columns=df.columns)
    for i in df.columns:
        for j in df.columns:
            if ratios.loc[i,j] == 0 and i != j:
                out.loc[i,j] = coint(df[i], df[j])
                out.loc[j,i] = out.loc[j,i]


class KalmanFilter(Strategy):
    '''Predicts fairs based on correlated movements between pairs.
    All inputs should be cointegrated.'''

    def __init__(self, correlation_window_size, movement_half_life):
        self.moving_prices_history = None
        self.correlation_window_size = correlation_window_size
        self.moving_prices = Ema(movement_half_life)
        self.moving_volumes = Ema(correlation_window_size / 2)
        self.prev_prediction = None

    def step(self, frame):
        prices = frame['price']
        volumes = frame['volume']
        if self.moving_prices_history is None:
            self.moving_prices_history = RingBuffer(
                self.correlation_window_size, dtype=(np.float64, len(prices.index)))

        if self.prev_prediction is None:
            self.prev_prediction = self.null_estimate(prices)

        self.moving_prices.step(prices)
        self.moving_volumes.step(volumes)

        if not self.moving_prices.ready:
            return self.null_estimate(prices)

        self.moving_prices_history.append(self.moving_prices.value)

        if len(self.moving_prices_history) < self.correlation_window_size:
            return self.null_estimate(prices)

        df = pd.DataFrame(self.moving_prices_history, columns=prices.index)
        diffs = df.diff().iloc[1:]
        diff = Gaussian(diffs.iloc[-1], diffs.cov())
        # Could also calculate diff from the raw price movements but using smoothed movements
        # for diff seems to improve RoR

        
        stddevs = df.std()
        corr = df.corr()
#         r2 = corr * corr
#         cov = df.cov()
        deltas = prices - df.mean()
#         volume_signals = np.sqrt(self.moving_volumes.value * self.prev_prediction.mean)
#         volume_factor = np.max(volume_signals) / volume_signals
#         correlated_predictions = []
#         for pair, delta in deltas.items():
#             correlated_mean = corr[pair] * stddevs * delta / stddevs[pair]
#             correlated_covariance = cov.div(r2[pair], axis=1) * volume_factor[pair]# * np.abs(delta) / stddevs[pair]
#             correlated_predictions.append(Gaussian(correlated_mean, correlated_covariance))
#         predicted_deltas = Gaussian.intersect(correlated_predictions)
#         print(predicted_deltas)
        predicted_delta_means = corr.mul(deltas, axis=0).mul(stddevs, axis=1).div(stddevs, axis=0)
        predicted_delta_variances = np.abs(df.cov().mul(stddevs, axis=1).div(stddevs, axis=0)) / (corr * corr + 1e-100)
        predicted_deltas = Gaussian.intersect([Gaussian(
            predicted_delta_means.loc[i], predicted_delta_variances.loc[i]) for i in prices.index])
        new_prediction = (self.prev_prediction + diff) & (predicted_deltas + df.mean())
        self.prev_prediction = new_prediction
        return new_prediction

analyze(run(KalmanFilter(correlation_window_size = 64, movement_half_life = 1), data_15min.tail(2000), fees=0.00075, min_edge=0.001))

In [ ]:
from strategy import CointegratorStrategy

analyze(run(CointegratorStrategy(cointegration_window_size = 128), data_15min.tail(1500)))

In [ ]:
import numpy as np
import pandas as pd
from numpy_ringbuffer import RingBuffer
import pickle

from research.strategy.base import Strategy
from trader.util.stats import Ema, Gaussian
from trader.util.linalg import orthogonal_projection

class Cointegrator(Strategy):
    def __init__(self, coint_file):
        coint = pickle.load(open(coint_file, 'rb'))
        self.A = coint['cointegrated_vectors']
        self.historical_mean_prices = coint['mean_prices']
        self.covs = coint['residual_covariances']
        self.prev_prices = None
        
    def __cointegrated_fairs(self, prices, base_prices):
        prices_norm = prices / base_prices - 1
        fair_means = [prices - orthogonal_projection(prices_norm, x) * base_prices for x in self.A.values]
        fairs = [Gaussian(mean, self.covs[i] * len(fair_means)) for i, mean in enumerate(fair_means)]
        return Gaussian.intersect(fairs)

    def step(self, frame):
        # TODO check that frame contains the currencies that the model was trained on
        prices = frame["price"]
#         assert np.array_equal(
#             prices.index,
#             np.array(["BTC_USDT", "XRP_USDT", "ETH_USDT", "LTC_USDT", "NEO_USDT", "EOS_USDT"]),
#         )
        
        if self.prev_prices is None:
            self.prev_prices = prices
            return self.null_estimate(prices)
            
        step_fairs = self.__cointegrated_fairs(prices, self.prev_prices)
        fairs = self.__cointegrated_fairs(prices, self.historical_mean_prices)
        prediction = fairs & step_fairs
        
        return prediction

analyze(run(Cointegrator("coint.p"), data_15min.tail(2000), fees=0.002, min_edge=0))

In [ ]:
import numpy as np
import pandas as pd
from numpy_ringbuffer import RingBuffer
import pickle

from research.strategy.base import Strategy
from trader.util.stats import Ema, Gaussian
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from trader.util.linalg import orthogonal_projection, hyperplane_projection

class LiveCointegrator(Strategy):
    def __init__(self, window_size, cointegration_frequency=4):
        self.window_size = window_size
        self.cointegration_period = window_size // cointegration_frequency
        self.sample_counter = 0
        self.price_history = None
        self.A = None
        self.base_prices = None
        self.covs = None
        self.prev_prediction = None

    def step(self, frame):
        prices = frame["price"]
        
        if self.price_history is None:
            self.price_history = RingBuffer(self.window_size, dtype=(np.float64, len(prices.index)))
            
        if self.prev_prediction is None:
            self.prev_prediction = self.null_estimate(prices)
            
        self.price_history.append(prices)
        
        if len(self.price_history) < self.window_size:
            return self.null_estimate(prices)
        
        if self.sample_counter == 0:
            P = pd.DataFrame(self.price_history, columns=prices.index)
            self.base_prices = P.mean()
            P_norm = P / self.base_prices - 1

            c = coint_johansen(P_norm, det_order=-1, k_ar_diff=1)
            
#             tail = P.tail(self.cointegration_period)
#             c_tail = coint_johansen(tail - tail.mean(), det_order=-1, k_ar_diff=1)
#             tail_cointegrated = np.any((c_tail.lr1 > c_tail.cvt[:,1]) * (c_tail.lr2 < c_tail.cvm[:,2]))

            significant_results = (c.lr1 > c.cvt[:,1]) * (c.lr2 < c.cvm[:,2])
            if np.any(significant_results): # and tail_cointegrated:
                self.A = pd.DataFrame(c.evec[:, significant_results].T, columns=prices.index)
                covs = []
                for a in self.A.values:
                    prediction_cov = orthogonal_projection(P_norm, a).cov()
                    w_pred, v_pred = np.linalg.eigh(prediction_cov)
                    w_sam, v_sam = np.linalg.eigh(P_norm.cov())
#                     print(w_pred)
#                     print(v_pred)
#                     print(w_sam)
#                     print(v_sam)
                    w_reg = w_pred + w_sam
                    regularized_cov = v_pred @ np.diag(w_reg) @ v_pred.T
#                     u_pred, s_pred, vh_pred = np.linalg.svd(prediction_cov)
#                     u_sam, s_sam, vh_sam = np.linalg.svd(P_norm.cov())
#                     s_reg = s_pred + s_sam
# #                     print(s_pred)
# #                     print(s_sam)
# #                     print(prediction_cov)
# #                     print(u_pred @ np.diag(s_pred) @ vh_pred)
#                     regularized_cov = u_pred @ np.diag(s_reg) @ vh_pred
#                     regularized_cov = pd.DataFrame(regularized_cov, index=P.columns, columns=P.columns)
# #                     print(regularized_cov)
                    covs.append(regularized_cov)
                self.covs = covs
            else:
                self.A = None
                self.covs = None
            
        self.sample_counter -= 1
        self.sample_counter %= self.cointegration_period
        
        if self.A is None:
            return self.null_estimate(prices)
        
        prices_norm = prices / self.base_prices - 1
        fair_means = [(hyperplane_projection(prices_norm, a) + 1) * self.base_prices for a in self.A.values]
        prediction = Gaussian(fair_means[0], self.covs[0])
#         fairs = [Gaussian(mean, self.covs[i]) for i, mean in enumerate(fair_means)]
#         prediction = Gaussian.intersect(fairs)
        return prediction

analyze(run(LiveCointegrator(window_size=128), data_15min.tail(2000), fees=0.001, min_edge=0))

In [ ]:
import numpy as np
import pandas as pd
from numpy_ringbuffer import RingBuffer
import pickle

from research.strategy.base import Strategy
from trader.util.stats import Ema, Gaussian
from trader.util.linalg import orthogonal_projection, hyperplane_projection
from statsmodels.tsa.stattools import coint

def plot_gaussian(x, y, gaussian):
    z = gaussian.pdf(np.dstack(np.meshgrid(x,y)))
    plt.pcolormesh(x, y, z)
    plt.show()

class LivePairCointegrator(Strategy):
    def __init__(self, window_size, cointegration_frequency=4):
        self.window_size = window_size
        self.cointegration_period = window_size // cointegration_frequency
        self.sample_counter = 0
        self.price_history = None

    def step(self, frame):
        prices = frame["price"]
        
        if self.price_history is None:
            self.price_history = RingBuffer(self.window_size, dtype=(np.float64, len(prices.index)))
            
        self.price_history.append(prices)
        
        if len(self.price_history) < self.window_size:
            return self.null_estimate(prices)
        
        df = pd.DataFrame(self.price_history, columns=prices.index)
        mean = df.mean()
        deltas = df / mean - 1
        stddev = deltas.std() + 1e-10
        regression_slope = deltas.corr().mul(stddev, axis=1).div(stddev, axis=0)
        delta = deltas.iloc[-1]
        
        if self.sample_counter == 0:
            coint_p = pd.DataFrame(0, index=prices.index, columns=prices.index)
            prediction_covs = {}
            for pair_i in prices.index:
                for pair_j in prices.index:
                    if pair_i >= pair_j:
                        continue
                    p = coint(df[pair_i], df[pair_j])[1]
                    coint_p.loc[pair_i, pair_j] = p
                    coint_p.loc[pair_j, pair_i] = p
                    deltas_ij = deltas[[pair_i, pair_j]]
                    regression_vector = [regression_slope.loc[pair_i][pair_j], -1]
                    prediction_cov = orthogonal_projection(deltas_ij, regression_vector).cov()
                    prediction_eigenvalues, prediction_eigenvectors = np.linalg.eig(prediction_cov)
                    sample_eigenvalues, sample_eigenvectors = np.linalg.eig(deltas_ij.cov())
                    regularized_eigenvalues = prediction_eigenvalues + sample_eigenvalues
                    regularized_cov = prediction_eigenvectors @ np.diag(regularized_eigenvalues) @ prediction_eigenvectors.T
                    regularized_cov = pd.DataFrame(regularized_cov, index=deltas_ij.columns, columns=deltas_ij.columns)
#                     prediction = Gaussian(pd.Series(0, index=deltas_ij.columns), prediction_cov)
#                     base = Gaussian(pd.Series(0, index=deltas_ij.columns), deltas_ij.cov())
#                     regularized = Gaussian(pd.Series(0, index=deltas_ij.columns), regularized_cov)
#                     prediction_cov = deltas_ij.cov()
                    prediction_covs[pair_i, pair_j] = regularized_cov
                    prediction_covs[pair_j, pair_i] = regularized_cov[::-1].T[::-1]
#                     xrange = deltas[pair_i].max() - deltas[pair_i].min()
#                     yrange = deltas[pair_j].max() - deltas[pair_j].min()
#                     x = np.linspace(-xrange, xrange)
#                     y = np.linspace(-yrange, yrange)
#                     print(prediction_cov)
#                     print(deltas_ij.cov())
#                     print(regularized_cov)
#                     plot_gaussian(x, y, prediction)
#                     plot_gaussian(x, y, base)
#                     plot_gaussian(x, y, regularized)
#                     print('abs')
#                     df[[pair_i, pair_j]].plot.scatter(x=pair_i,y=pair_j)
#                     plt.show()
                    
            self.coint_f = (coint_p * coint_p * 900).clip(1) # discount predictions for p values > .03
            self.prediction_covs = prediction_covs
            
        self.sample_counter -= 1
        self.sample_counter %= self.cointegration_period
        
        fairs = []
        base_cov = df.cov()
#         print('')
        for pair_i in prices.index:
            for pair_j in prices.index:
                if pair_i >= pair_j:
                    continue
                regression_vector = [regression_slope.loc[pair_i][pair_j], -1]
#                 residuals = orthogonal_projection(deltas[[pair_i, pair_j]], regression_vector)
                fair_delta_mean = hyperplane_projection(delta[[pair_i, pair_j]], regression_vector)
                fair_cov = self.prediction_covs[pair_i, pair_j] * self.coint_f.loc[pair_i, pair_j]
                fair = Gaussian(fair_delta_mean, fair_cov)
#                 print(fair)
#                 x = np.linspace(deltas[pair_i].min(), deltas[pair_i].max()) - deltas[pair_i].mean()
#                 y = np.linspace(deltas[pair_j].min(), deltas[pair_j].max()) - deltas[pair_j].mean()
#                 plot_gaussian(x, y, fair)
#                 plot_gaussian(x, y, Gaussian(pd.Series(0, index=[pair_i, pair_j]), residuals.cov()))
#                 plot_gaussian(x, y, Gaussian(pd.Series(0, index=[pair_i, pair_j]), deltas[[pair_i, pair_j]].cov()))
                fairs.append(fair)
        fair = (Gaussian.intersect(fairs) + 1) * mean
#         print(fair)
#         print(base_cov)
#         x = np.linspace(df['BTC_USDT'].min(), df['BTC_USDT'].max())
#         y = np.linspace(df['ETH_USDT'].min(), df['ETH_USDT'].max())
#         print(prices)
#         print(fair)
#         print(fair[['BTC_USDT', 'ETH_USDT']])
#         plot_gaussian(x, y, fair[['BTC_USDT', 'ETH_USDT']])
#         print(fair - prices)
        return fair

analyze(run(LivePairCointegrator(window_size=128), data_15min.tail(500), fees=0.001, min_edge=0))

In [ ]:
analyze(run(CointegratorStrategy(cointegration_window_size = 16), data_15min))

In [ ]:
analyze(run(KalmanFilterStrategy(
    correlation_window_size = 480,
    movement_half_life = 1
), tail_data(data_min, 10000), fees = 0.002))

In [ ]:
from strategy import CombinedStrategy

analyze(run(CombinedStrategy([
    KalmanFilterStrategy(correlation_window_size = 60, movement_half_life = 3),
    CointegratorStrategy(cointegration_window_size = 16)
]), data_15min))

In [ ]:
analyze(run(CointegratorStrategy(cointegration_window_size = 512), data_5min))

In [ ]:
analyze(run(KalmanFilterStrategy(correlation_window_size = 165, movement_half_life = 70), data_5min))

In [ ]:
analyze(run(CombinedStrategy([
    KalmanFilterStrategy(correlation_window_size = 16, movement_half_life = 8),
    CointegratorStrategy(cointegration_window_size = 64)
]), tail_data(data_min, 1500)))

In [ ]:
import random

def find_best_window_sizes(data, n):
    points = []
    best = None
    best_ror = 0
    for _ in range(n):
        movement_half_life = random.expovariate(1) * 15
#         window_ratio = random.uniform(1, 10)
#         window_size = max(3, int(movement_half_life * window_ratio))
#         movement_half_life = 2
        window_size = int(random.expovariate(1) * 100) + 400
#         window_size = int(random.expovariate(1) * 30) + 3
#         window_size = 4
#         window_size = 32
        print('Trying window_size: {0} and half_life: {1}'.format(window_size, movement_half_life))
        ror = analyze(run(KalmanFilter(window_size, movement_half_life), data, fees = 0.002), plot=False)
        print('  RoR: {0}'.format(ror))
        point = { 'window_size': window_size, 'half_life': movement_half_life, 'RoR': ror }
        points.append(point)
        if ror > best_ror:
            best = point
            best_ror = ror
    print('Best found:')
    print(best)
    pd.DataFrame(points).plot.scatter('window_size', 'half_life', c='RoR', colormap='jet')
    
find_best_window_sizes(tail_data(data_min, 1500), 25)